In [1]:
import tensorflow as tf
#from tensorflow import keras
import keras # if installed directly
import numpy as np
#!pip install livelossplot --quiet # library to plot training logs
#from livelossplot import PlotLossesKeras

"tf", tf.__version__, "Keras", keras.__version__

Using TensorFlow backend.


('tf', '1.9.0', 'Keras', '2.2.2')

Loading in a dataset:

In [30]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

# simple norm
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [31]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

In [32]:
#data augmentation
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.05,
    height_shift_range=0.05,
    horizontal_flip=True,
    )
datagen.fit(x_train, augment=False)

In [39]:
def make_model(dropout=0.0):
    model = Sequential()

    model.add(Conv2D(28, (3, 3), padding='same', activation='relu', 
                   input_shape=(28, 28, 1)))
    #model.add(BatchNormalization())
    model.add(Conv2D(28, (3, 3), activation='relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout > 0: model.add(Dropout(dropout))

    model.add(Conv2D(56, (3, 3), padding='same', activation='relu'))
    #model.add(BatchNormalization())
    model.add(Conv2D(56, (3, 3), activation='relu'))
    #model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    if dropout > 0: model.add(Dropout(dropout))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    if dropout > 0: model.add(Dropout(0.5))

    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer='Adam',
               loss = 'sparse_categorical_crossentropy',
               metrics=['accuracy'])

    return model

In [ ]:
dropouts = [0.0, 0.2, 0.4, 0.6]
history = []

for dropout in dropouts:
    model = make_model(dropout)
    print(f"Running model with dropout {dropout}")
    
    hist = model.fit(x_train, y_train, 
                    epochs=20, validation_data=(x_test, y_test), 
                    callbacks=[EarlyStopping(patience=4)])
    
    print(f"Model with dropout {dropout} finished with {max(hist.history['val_acc'])} accuracy")
    history.append(hist)

Running model with dropout 0.0
Train on 60000 samples, validate on 10000 samples
Epoch 1/20
60000/60000 [==============================] - 304s 5ms/step - loss: 0.1027 - acc: 0.9680 - val_loss: 0.0402 - val_acc: 0.9866
Epoch 2/20
60000/60000 [==============================] - 269s 4ms/step - loss: 0.0358 - acc: 0.9893 - val_loss: 0.0365 - val_acc: 0.9885
Epoch 3/20
29728/60000 [=============>................] - ETA: 1:55 - loss: 0.0252 - acc: 0.9925

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

colors = ["C"+str(i) for i in range(5)]
fig = plt.figure(figsize=(14,10))
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')

for dropout, hist, color in zip(dropouts, history, colors):
  plt.plot(hist.history['acc'], label=f"dropout_{dropout:.1f} ", ls="dashed", lw=2, alpha=0.6, c=color)
  plt.plot(hist.history['val_acc'], label=f"dropout_{dropout:.1f} Acc{max(hist.history['val_acc']):.2f}", ls="solid", lw=3, alpha=0.8, c=color)
  
plt.legend()
plt.show()

In [ ]:
fig = plt.figure(figsize=(14,10))
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')

for dropout, hist in zip(dropouts, history):
  plt.plot(hist.history['loss'], label=f"dropout_{dropout:.1f}", ls="dashed", lw=2, alpha=0.5)
  plt.plot(hist.history['val_loss'], label=f"dropout_{dropout:.1f}", ls="solid", lw=3, alpha=0.8)
  
plt.legend()
plt.show()